In [22]:
import numpy as np
import nn_tools
from torch.nn import MSELoss
from torch.optim import RMSprop
from torch import from_numpy
from torch.utils.data import DataLoader, TensorDataset

In [13]:
def perform_one_epoch(model, train_loader, criterion, optimizer, device='cpu'):
    model.to(device)
    model.train()
    
    total_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    return total_loss/len(train_loader)

In [14]:
def prepare_data(states, targets, batch_size):
    dataset = TensorDataset(from_numpy(states), from_numpy(targets))
    trainloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return trainloader

In [ ]:
def split_D_layers(D, nb_layers):
    datasets = [[] for k in range(nb_layers)]
    for x, b, c, t in D:
        datasets[t].append((np.concatenate([x, b]), c))
    return datasets

def optimiers_list(layers_Q):
    return [RMSprop(model.parameters(), lr = 0.83,alpha=0.98, momentum=0.82) for model in layers_Q]

def one_epoch_layers(layers_Q, D_by_layers, opti_list, loss = MSELoss, batch_size = 50, device = "cpu"):
    """
    layers_Q: liste des modèles Q
    D_by_layer : jeu d'entraînement
                Format : iterable de taille len(layers_Q)
                Cahque élément D_by_layer[k] correspond au jeu d'entraînement pour la ke couche.
                Format de D_by_layer[k] : itérable de taille 2
                D_by_layer[k][0] est un array de shape (n_samples, n_state_features) contenant les états
                D_by_layer[k][1] est un array de shape (n_samples, 36) contenant les targets
    opti_list : list of optimizers. They can be generated by optimiers_list function
    """
    loss_by_layers = []
    for t, d_t in enumerate(D_by_layers):
        train_loader = prepare_data(d_t[0], d_t[1], batch_size)
        loss_by_layers[t] = perform_one_epoch(layers_Q[t], train_loader, loss, opti_list[t], device)
    return loss_by_layers

SyntaxError: non-default argument follows default argument (4095152629.py, line 10)

In [17]:
import pandas as pd
df = pd.read_csv("data/slice0.csv", index_col = 0)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175
0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.60,0.64,0.52,0.56,0.84,0.92,0.92,1.00,0.76,0.88,0.80,0.84,0.92,0.72,0.76,0.76,0.80,0.80,0.68,0.72,0.72,0.76,0.84,0.64,0.68,0.68,0.68,0.72,0.60,0.64,0.64,0.64,0.68,0.56,0.60,0.60,0.60,0.64,0.52,0.56
1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.64,0.60,0.60,0.60,0.84,0.84,0.96,1.00,0.88,0.88,0.76,0.92,0.88,0.84,0.84,0.72,0.84,0.76,0.76,0.76,0.68,0.76,0.72,0.72,0.72,0.64,0.72,0.68,0.68,0.68,0.60,0.68,0.64,0.64,0.64,0.56,0.64,0.60,0.60,0.60
2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.52,0.64,0.60,0.56,0.88,0.76,0.76,1.00,0.88,0.80,0.72,0.68,0.96,0.84,0.76,0.68,0.64,0.88,0.80,0.72,0.64,0.60,0.76,0.72,0.68,0.60,0.60,0.72,0.68,0.64,0.56,0.56,0.68,0.64,0.60,0.52,0.52,0.64,0.60,0.56
3,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.44,0.44,0.52,0.48,0.56,0.68,0.52,0.52,0.72,0.64,0.68,0.48,0.48,0.72,0.60,0.68,0.48,0.48,0.72,0.52,0.60,0.48,0.48,1.00,0.52,0.52,0.44,0.44,0.68,0.48,0.52,0.44,0.44,0.52,0.48,0.48,0.44,0.44,0.52,0.48
4,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.52,0.48,0.52,0.52,0.88,1.00,0.76,0.68,0.72,0.72,0.88,0.72,0.64,0.68,0.68,0.80,0.68,0.60,0.64,0.64,0.76,0.64,0.56,0.60,0.60,0.72,0.60,0.52,0.56,0.56,0.68,0.56,0.52,0.56,0.56,0.64,0.52,0.48,0.52,0.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.64,0.52,0.56,0.56,0.88,0.76,1.00,0.72,0.84,0.84,0.72,0.92,0.68,0.76,0.76,0.68,0.80,0.64,0.72,0.72,0.64,0.76,0.60,0.68,0.68,0.60,0.72,0.56,0.64,0.64,0.56,0.68,0.52,0.60,0.60,0.52,0.64,0.52,0.56,0.56
9996,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.52,0.60,0.60,0.56,0.88,1.00,0.68,0.88,0.88,0.80,0.92,0.64,0.80,0.84,0.72,0.80,0.60,0.76,0.76,0.68,0.76,0.56,0.68,0.72,0.64,0.72,0.56,0.64,0.68,0.60,0.68,0.52,0.60,0.64,0.56,0.64,0.52,0.60,0.60,0.56
9997,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.56,0.56,0.56,0.56,0.76,0.84,0.80,0.84,0.84,0.88,0.84,0.76,0.76,0.76,0.88,0.84,0.68,0.68,0.68,1.00,0.80,0.64,0.64,0.64,0.68,0.68,0.60,0.60,0.60,0.64,0.64,0.56,0.56,0.56,0.60,0.60,0.56,0.56,0.56,0.56
9998,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.56,0.60,0.48,0.56,0.72,0.88,0.84,0.88,0.68,0.84,0.88,0.84,0.88,0.60,0.80,0.88,0.80,0.84,0.60,0.84,0.88,0.72,1.00,0.56,0.64,0.84,0.64,0.68,0.52,0.60,0.68,0.60,0.64,0.52,0.56,0.64,0.56,0.60,0.48,0.56


In [26]:
D = []
for k in range(5):
    if k==0:
        D.append([np.array(df.iloc[:,:52]), np.array(df.iloc[:,-36:])])
    else:
        if k%2 == 0:
            D.append([np.array(df.iloc[:,:52]), np.array(df.iloc[:,-36:])])
        if k%2 == 1:
            D.append([np.array(df.iloc[:,52:104]), np.array(df.iloc[:,-36:])])

In [25]:
Q_list = nn_tools.define_Q(5)

In [27]:
one_epoch_layers(Q_list, D)

TypeError: __init__() missing 1 required positional argument: 'params'